In [1]:
import json
import os
import pandas as pd
import more_itertools as mit
from collections import Counter
import collections 
import pickle
import time
import copy


In [2]:
run1="AMT_First_batch_03032021"
run2="AMT_2nd_batch_09032021"
run3="Batch_4360660_100_200_results"
run4="Batch_4362039_200_500_results"
run5="Batch_4363418_500_1100_results_v5"
run6="Long_Batch_4364840_0_100_results_v3"
run7="Long_Batch_4367993_100_300_results_v2"
run8="Long_Batch_4371552_300_500_results_v3"
run9="Long_Batch_4374976_500_700_results_v3"
run10="Batch_4377561_1100_1600_results_v2"
run11="Batch_4383845_1600_1900_results"

filename = run11

amt_results = pd.read_csv("amt_results/raw_csv/" + filename + ".csv")
amt_results['Input.dream_id_name'] = amt_results['Input.dream_id_name'].astype('str')
amt_results['Input.clean_description'] = amt_results['Input.clean_description'].astype('str') 
amt_results.dtypes

HITId                           object
HITTypeId                       object
Title                           object
Description                     object
Keywords                        object
Reward                          object
CreationTime                    object
MaxAssignments                   int64
RequesterAnnotation             object
AssignmentDurationInSeconds      int64
AutoApprovalDelayInSeconds       int64
Expiration                      object
NumberOfSimilarHITs            float64
LifetimeInSeconds              float64
AssignmentId                    object
WorkerId                        object
AssignmentStatus                object
AcceptTime                      object
SubmitTime                      object
AutoApprovalTime                object
ApprovalTime                   float64
RejectionTime                  float64
RequesterFeedback              float64
WorkTimeInSeconds                int64
LifetimeApprovalRate            object
Last30DaysApprovalRate   

# Convert HTML encoded characters back to normal

In [3]:
#Remove leading and trailing apostrophe
amt_results['Input.clean_description'] = amt_results['Input.clean_description'].str.strip("'")
amt_results['Input.dream_id_name'] = amt_results['Input.dream_id_name'].str.strip("'")

amt_results['Input.clean_description'] = amt_results['Input.clean_description'].str.replace('&#44', ',')
amt_results['Input.clean_description'] = amt_results['Input.clean_description'].str.replace('&#39', '\'')
amt_results['Input.clean_description'] = amt_results['Input.clean_description'].str.replace('&#34', '\"')
amt_results['Input.dream_id_name'] = amt_results['Input.dream_id_name'].str.replace('&#44', ',')
amt_results['Input.dream_id_name'] = amt_results['Input.dream_id_name'].str.replace('&#39', '\'')
amt_results['Input.dream_id_name'] = amt_results['Input.dream_id_name'].str.replace('&#34', '\"')

In [4]:
amt_results['Input.dream_id_name'].head()

0                 298396 - 5 pack Rove Cartridges 1/2g
1                 298396 - 5 pack Rove Cartridges 1/2g
2    428053 - Platinum and Top Shelf Gorilla Glue 3 LB
3    428053 - Platinum and Top Shelf Gorilla Glue 3 LB
4     175003 - SAMPLE! .5g OF HIGH QUALITY BUBBLE HASH
Name: Input.dream_id_name, dtype: object

In [5]:
amt_results['Input.clean_description'].head()

0    5 pack of Rove 1 / 2 gram cartridges. Save 5 p...
1    5 pack of Rove 1 / 2 gram cartridges. Save 5 p...
2    Description Platinum and Top Shelf Gorilla Glu...
3    Description Platinum and Top Shelf Gorilla Glu...
4    SAMPLE SALE. Reviews would be appreciated. Thi...
Name: Input.clean_description, dtype: object

# Remove empty annotations, since those were from a script, but I had to accept them to get Masters work with me.

In [6]:
amt_results.shape

(898, 32)

In [7]:
amt_results = amt_results[amt_results['RejectionTime'].isna()]
amt_results.shape

(898, 32)

# Majority vote Aggregation Mechanism



In [8]:
def calculate_annotated_ranges(taskAnswers):
    answer= json.loads(taskAnswers)
    marked_chars = []
    if len(answer) == 0 or len(answer[0].keys()) == 0:
        print('row was empty') 
    else:
        key= list(answer[0].keys())[0] 
        for ent in answer[0][key]['entities']:
            #entity_tuples.append((ent['startOffset'], ent['endOffset']))
            marked_chars.extend(range(ent['startOffset'], ent['endOffset']))
        
    return marked_chars
amt_results['marked_chars'] = amt_results['Answer.taskAnswers'].apply(lambda answer: calculate_annotated_ranges(answer))   


In [9]:
def aggregate_ranges(series):
    counter = collections.Counter() 
    for annotation in series:
        counter.update(Counter(annotation)) 

    threshold = len(series) / 2
    mv_dict= {key:val for key, val in dict(counter).items() if val >= threshold}
    final_ranges=[]
    
    for group in mit.consecutive_groups(list(mv_dict.keys())):
        group_list = list(group)
        final_ranges.append((group_list[0], group_list[-1]))
    return final_ranges

#test = amt_results[amt_results.HITId == '3BFF0DJK8XC0JVZWP5F1P36TQW6STQ']
#test.groupby('HITId').marked_chars.agg(aggregate_ranges)
mv_annotations = amt_results.groupby('HITId').marked_chars.agg(aggregate_ranges)

# Print  current annotations

In [10]:
for i, row in amt_results.iterrows():
    print('Row: ', i, ' description: ', row['Input.clean_description'])
    
    annotations = mv_annotations[row['HITId']]
    
    for (start, end) in annotations:
        print('start: ', start, ' end: ', end, ' text: ', row['Input.clean_description'][start:end+1])

Row:  0  description:  5 pack of Rove 1 / 2 gram cartridges. Save 5 plus free shipping off the individual price. Please specify flavors or we will choose for you. - APE indica OG indica Kush indica SHERBET hybrid Dream hybrid Cookies hybrid HAZE sativa. Waui sativa GLUE hybrid.
start:  142  end:  154  text:  APE indica OG
start:  156  end:  166  text:  indica Kush
start:  169  end:  188  text:  ndica SHERBET hybrid
start:  190  end:  201  text:  Dream hybrid
start:  203  end:  216  text:  Cookies hybrid
start:  218  end:  228  text:  HAZE sativa
start:  231  end:  241  text:  Waui sativa
start:  243  end:  253  text:  GLUE hybrid
start:  155  end:  155  text:   
start:  167  end:  168  text:   i
Row:  1  description:  5 pack of Rove 1 / 2 gram cartridges. Save 5 plus free shipping off the individual price. Please specify flavors or we will choose for you. - APE indica OG indica Kush indica SHERBET hybrid Dream hybrid Cookies hybrid HAZE sativa. Waui sativa GLUE hybrid.
start:  142  end

start:  27  end:  30  text:  OXYS
Row:  57  description:  Back in stock on authentic OXYS. Get yours before they are all gone. Manufacturer - MALLINCKRODT. Quantity - 2 pills.
start:  27  end:  30  text:  OXYS
Row:  58  description:  14G OF OUR SPEEDOFFER OPEN TO ALL CUSTOMERS WITH MORE THAN 5 SALESESCROW SERVICEORDERS DISPATCHED THROUGHOUT THE WEEK , MOND - SAT HIGHEST AVAILABLE STEALTH IS USEDONE SAMPLES OFFER PER CUSTOMERPLEASE ADD NAMES TO ADDRESSALSO ADD DESINATION NATION.
Row:  59  description:  14G OF OUR SPEEDOFFER OPEN TO ALL CUSTOMERS WITH MORE THAN 5 SALESESCROW SERVICEORDERS DISPATCHED THROUGHOUT THE WEEK , MOND - SAT HIGHEST AVAILABLE STEALTH IS USEDONE SAMPLES OFFER PER CUSTOMERPLEASE ADD NAMES TO ADDRESSALSO ADD DESINATION NATION.
Row:  60  description:  14G OF OUR SPEEDOFFER OPEN TO ALL CUSTOMERS WITH MORE THAN 5 SALESESCROW SERVICEORDERS DISPATCHED THROUGHOUT THE WEEK , MOND - SAT HIGHEST AVAILABLE STEALTH IS USEDONE SAMPLES OFFER PER CUSTOMERPLEASE ADD NAMES TO ADDRES

start:  146  end:  149  text:  MDMA
start:  192  end:  197  text:  2C - B
start:  287  end:  289  text:  MDA
start:  295  end:  300  text:  2C - B
start:  480  end:  485  text:  2C - B
start:  547  end:  549  text:  MDA
Row:  159  description:  You asked for it. NEXUS FOREVER special , fasten your seat belts and get ready for new dimensions. You re getting 100 mgr capsule of INTERSTELLAR MDMA and a 20 mgr capsule of top shelf finest 2C - B. Many of the reports that have come in over the years have mentioned the combination of MDA and 2C - B. The most successful reports have followed a program in which the two drugs are not used at the same time , nor even too closely spaced. It appears that the optimum time for the 2C - B is at , or just before , the final baseline recovery of the MDA. Keep in mind that both will potentiate each other significantly , Experienced ONLY ALERT. Enjoy responsibly and have a great one.
start:  146  end:  149  text:  MDMA
start:  192  end:  197  text:  2C - B

start:  31  end:  38  text:  cocaine 
start:  76  end:  80  text:  coke 
start:  151  end:  157  text:  cocaine
start:  322  end:  329  text:  Cocaine 
Row:  264  description:  1g Social Cocaine. Good social cocaine comes straight from the source. Same coke as i ve been selling on the darknet since day one. Better than street cocaine. Will mostly arrive in rock form. Good for a night out with friends. Free Domestic delivery. United Kingdom and Northern Ireland delivery only. Thanks for looking Cocaine Cowboy.
start:  10  end:  16  text:  Cocaine
start:  31  end:  38  text:  cocaine 
start:  76  end:  80  text:  coke 
start:  151  end:  157  text:  cocaine
start:  322  end:  329  text:  Cocaine 
Row:  265  description:  Welcome to YD `s new Critical Kush. Try it now. As usual YD will ship that fast and safe to your location ,. please read our profil before ordering. WEED , HASH , SPEED , COCAINE , MDMA and MORE YOURDEALER is here for you.
start:  21  end:  33  text:  Critical Kush
start

start:  144  end:  161  text:  oxycodone Percocet
start:  164  end:  172  text:  Oxycodone
start:  180  end:  185  text:  opioid
start:  207  end:  212  text:  opioid
start:  236  end:  243  text:  narcotic
start:  246  end:  258  text:  Acetaminophen
start:  321  end:  329  text:  oxycodone
start:  351  end:  363  text:  acetaminophen
start:  369  end:  377  text:  oxycodone
start:  0  end:  7  text:  PERCOCET
Row:  340  description:  This is a high THC dry-ice sifted hashish that is 100 solvent free. All capitulate gland THC , made from Girl Scout Cookies strain. Full flavor and extremely strong. 3 grams. 22 ADD - ON ITEMS. For an extra 22 you can get a CO2 cartridge or edible item added to your order regular price 30. Select USPS Plus Extra Item shipping option. - SHIPPING -. Please make sure your shipping info is accurate and correctly formatted. SAMPLE ADDRESS. JOE DRAKE 585 MAIN ST APT 284 CHICAGO , IL 48392. Due to security issues and high order volumes , I do not provide tracki

...

start:  46  end:  49  text:  weed
Row:  582  description:  112g QP Bruce Banner Indoor organically grown weed. Testing at 231 THC. Passes dispensary test , No Bugs , No Mold , AAA -. High Res Pic. You can order on Trade Route as well. A much more secure site and it offers multisig transactions to ensure no exit scams can happen from site Admins. All orders guaranteed delivery or we ll reship another order to you. Orders are vacuum sealed then sealed in a mylar bag for additional security. All orders will be provided tracking data upon request by the customer.
start:  46  end:  49  text:  weed
Row:  583  description:  5000 x pink indian 220 mg high grade mdma. Nice hard pressed Great stealth.
start:  37  end:  40  text:  mdma
Row:  584  description:  5000 x pink indian 220 mg high grade mdma. Nice hard pressed Great stealth.
start:  37  end:  40  text:  mdma
Row:  585  description:  5000 x pink indian 220 mg high grade mdma. Nice hard pressed Great stealth.
start:  37  end:  40  text:  

start:  587  end:  612  text:  WHITE WIDOW X GELATO GUAVA
Row:  646  description:  This listing is 100 x 10 mg MSJ Diazepam. These will come loose as they come in 1000 piece tubs. Sealed in zip lock baggies and packaged further in foil Mylar bags for ultimate stealth and sent in padded envelopes. I take all benzos and oxycodones myself as they re top quality. These are genuine diazepam and I often take them after a long week working and I need a long heavy sleep. Lovely drowsy and wavey effect. Delivery is free with Royal Mail 1st class delivery. Allow 1 - 3 days for delivery as Royal Mail arent always the fastest br expect 80 chance of NDD if ordered before 2 pm.
start:  28  end:  39  text:  MSJ Diazepam
start:  226  end:  231  text:  benzos
start:  237  end:  246  text:  oxycodones
start:  297  end:  304  text:  diazepam
Row:  647  description:  This listing is 100 x 10 mg MSJ Diazepam. These will come loose as they come in 1000 piece tubs. Sealed in zip lock baggies and packaged fur

Row:  711  description:  - Blue Cheese strain. 35 g - Lovely strong odour - Flushed and cured - Potent - Same day dispatch b 4 330 GMT - Free delivery.
start:  2  end:  12  text:  Blue Cheese
Row:  712  description:  picture speaks for itself. WE DO NOT LIKE TO PUT PERCENTAGE BUT WE BELIEVE IT IS ABOVE 80. WE COOK FREEBASE AND WE GET BACK 83 GRAMS OF BASE FROM 10 GRAMS OF POWDER COCAINE. you get what is in the picture ,. WE DO NOT ADULTERATE OUR COCAINE. THATS FOR THE BIRDS. YOU GET IT STRAIGHT OFF OF THE BRICK.
start:  165  end:  171  text:  COCAINE
start:  233  end:  239  text:  COCAINE
Row:  713  description:  picture speaks for itself. WE DO NOT LIKE TO PUT PERCENTAGE BUT WE BELIEVE IT IS ABOVE 80. WE COOK FREEBASE AND WE GET BACK 83 GRAMS OF BASE FROM 10 GRAMS OF POWDER COCAINE. you get what is in the picture ,. WE DO NOT ADULTERATE OUR COCAINE. THATS FOR THE BIRDS. YOU GET IT STRAIGHT OFF OF THE BRICK.
start:  165  end:  171  text:  COCAINE
start:  233  end:  239  text:  COCAINE


start:  94  end:  97  text:  XTC 
Row:  790  description:  Genuine Pharmacy Eifelfango Testosterone Enanthate 250 mg / 1 ml X 10 Amps 2 Boxes of 5 Amps Each 1 ml Amp contains 250 mg Testosterone Enanthate. This listing is for 10 Amps.
start:  28  end:  49  text:  Testosterone Enanthate
start:  123  end:  144  text:  Testosterone Enanthate
start:  17  end:  27  text:  Eifelfango 
Row:  791  description:  Genuine Pharmacy Eifelfango Testosterone Enanthate 250 mg / 1 ml X 10 Amps 2 Boxes of 5 Amps Each 1 ml Amp contains 250 mg Testosterone Enanthate. This listing is for 10 Amps.
start:  28  end:  49  text:  Testosterone Enanthate
start:  123  end:  144  text:  Testosterone Enanthate
start:  17  end:  27  text:  Eifelfango 
Row:  792  description:  Genuine Pharmacy Eifelfango Testosterone Enanthate 250 mg / 1 ml X 10 Amps 2 Boxes of 5 Amps Each 1 ml Amp contains 250 mg Testosterone Enanthate. This listing is for 10 Amps.
start:  28  end:  49  text:  Testosterone Enanthate
start:  123  end:

start:  64  end:  71  text:  Suboxone
Row:  850  description:  Pseudoephedrine is a sympathomimetic drug of the phenethylamine and amphetamine chemical classes. It may be used as a nasal / sinus decongestant , as a stimulant , or as a wakefulness - promoting agent 2 in higher doses. 3. Highly competitive prices on Bulk , we can supply container loads.
start:  0  end:  14  text:  Pseudoephedrine
start:  49  end:  62  text:  phenethylamine
start:  68  end:  78  text:  amphetamine
Row:  851  description:  Pseudoephedrine is a sympathomimetic drug of the phenethylamine and amphetamine chemical classes. It may be used as a nasal / sinus decongestant , as a stimulant , or as a wakefulness - promoting agent 2 in higher doses. 3. Highly competitive prices on Bulk , we can supply container loads.
start:  0  end:  14  text:  Pseudoephedrine
start:  49  end:  62  text:  phenethylamine
start:  68  end:  78  text:  amphetamine
Row:  852  description:  Pseudoephedrine is a sympathomimetic drug of th

# Convert to Labelstudio format

In [11]:
amt_results['dream_id'] = amt_results['Input.dream_id_name'].str.split(' - ', n=1).str[0]
 # Only one split --> all other patterns are by accident --> n=1
amt_results['name'] = amt_results['Input.dream_id_name'].str.split(' - ', n=1).str[1]
amt_results['dream_id'].head()

0    298396
1    298396
2    428053
3    428053
4    175003
Name: dream_id, dtype: object

In [12]:
#Load data used for dataset creation
df_original = pd.read_pickle('intermediate_store/preprocessed_AMT_ads_V1.2.pickle')

In [13]:
with open('appen_results/label_studio_example.json') as json_file:
    label_studio = json.load(json_file)
label_studio[0]

{'completions': [{'created_at': 1610963709,
   'id': 641001,
   'lead_time': 11.385,
   'result': [{'from_name': 'label',
     'id': '9HDYDBGmSQ',
     'to_name': 'text',
     'type': 'labels',
     'value': {'end': 40,
      'labels': ['Drug'],
      'start': 30,
      'text': 'Lorazepam '}}]}],
 'data': {'Unnamed: 0': 11228,
  'category': 'DRUG',
  'description': 'You are purchasing one box of Lorazepam 5 mg with 60 pills packaged in flaskYou will receive exactly the box in the picture because I took the photo of the box I am sellingAll my medicines come from a pharmacy in Europe and are according to EMEA rules  ( European Medicines Agency )  . Our products are shipped in registered mail with tracking number and are carefully packaged .  For more information about the product please contact me',
  'keywords': "[('AA, Benzos', 30, 39)]",
  'lang': 'en',
  'market_name': 'DREAM',
  'name': 'Lorazepam 5 mg  60 pills  tavor 5 mg Ativan Temest',
  'subcategory': 'Drugs, Benzos',
  'vendor

In [14]:
'''
def get_result_json(annotation_list, text, hitID):
    result=[]
    indx = 0           
    for (startIdx, endIdx) in annotation_list:
        anno_dict = {'from_name': 'label',
                       'id': str(indx) + '-' + hitID,
                       'to_name': 'text',
                       'type': 'labels',
                       'value':{'end':endIdx+1,
                                    'labels': ['Drug'],
                                    'text':text[startIdx:endIdx+1],
                                    'start':startIdx}}
        result.append(anno_dict)
        indx = indx + 1
    #print('result:', result)
    return result

def create_labelstudio_data(agg_row):
                     
    original_row = df_original[df_original['dream_id'] == int(agg_row['dream_id'])].iloc[0]
    mv_annotation = mv_annotations[agg_row['HITId']]
    data={'dream_id':agg_row.dream_id, 'name':agg_row['name'], 'p_id':int(original_row.name), 
          'clean_description':agg_row['Input.clean_description'] }

    completions={'created_at': int(time.time()),
                'id':agg_row.HITId,
                'lead_time':10.00,        #I have no clue what lead time is.
                'result':get_result_json(mv_annotation, agg_row['Input.clean_description'], agg_row.HITId)
                }
    
    return {'completions':[completions], 'data':data, 'id':agg_row.HITId}


amt_results['labelstudio_data'] = amt_results.apply(lambda row: create_labelstudio_data(row), axis=1)
'''

"\ndef get_result_json(annotation_list, text, hitID):\n    result=[]\n    indx = 0           \n    for (startIdx, endIdx) in annotation_list:\n        anno_dict = {'from_name': 'label',\n                       'id': str(indx) + '-' + hitID,\n                       'to_name': 'text',\n                       'type': 'labels',\n                       'value':{'end':endIdx+1,\n                                    'labels': ['Drug'],\n                                    'text':text[startIdx:endIdx+1],\n                                    'start':startIdx}}\n        result.append(anno_dict)\n        indx = indx + 1\n    #print('result:', result)\n    return result\n\ndef create_labelstudio_data(agg_row):\n                     \n    original_row = df_original[df_original['dream_id'] == int(agg_row['dream_id'])].iloc[0]\n    mv_annotation = mv_annotations[agg_row['HITId']]\n    data={'dream_id':agg_row.dream_id, 'name':agg_row['name'], 'p_id':int(original_row.name), \n          'clean_descripti

In [15]:
def get_result_json(annotation_list, text, hitID):
    result=[]
    indx = 0           
    for (startIdx, endIdx) in annotation_list:
        anno_dict = {'from_name': 'label',
                       'id': str(indx) + '-' + hitID,
                       'to_name': 'text',
                       'type': 'labels',
                       'value':{'end':endIdx+1,
                                    'labels': ['Drug'],
                                    'text':text[startIdx:endIdx+1],
                                    'start':startIdx}}
        result.append(anno_dict)
        indx = indx + 1
    #print('result:', result)
    return result

def create_labelstudio_data(hitId, annotation):
    
    amt_row=amt_results[amt_results['HITId'] == hitId].iloc[0]

    original_row = df_original[df_original['dream_id'] == int(amt_row['dream_id'])].iloc[0]
    
    data={'dream_id':amt_row.dream_id, 'name':amt_row['name'], 'p_id':int(original_row.name), 
          'clean_description':amt_row['Input.clean_description'] }

    completions={'created_at': int(time.time()),
                'id':amt_row.HITId,
                'lead_time':10.00,        #I have no clue what lead time is.
                'result':get_result_json(annotation, amt_row['Input.clean_description'], hitId)
                }
    
    return {'completions':[completions], 'data':data, 'id':hitId}

final_data=[]
for hitId, annotation in mv_annotations.items():
    
    labelstudio_item = create_labelstudio_data(hitId, annotation)
    final_data.append(labelstudio_item)

In [16]:
#Write TO File
with open('amt_results/amt_final_agg/label_studio_' + filename + '_aggregated.json', 'w') as json_file:
    json.dump(final_data, json_file, indent='\t')